In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [2]:
from sympy import sqrt, cos, sin,  cosh,  sinh, Add
from sympy import Symbol, S, oo, Rational, Function, pi , factorial, O, symbols, lambdify, I
from sympy import Derivative, series, limit, Matrix, simplify, collect, solve, Poly, poly, diff, nsolve, transpose, factor_terms
from sympy import latex, init_printing
from sympy.core.numbers import NegativeOne, One, Zero
from sympy.printing.latex import print_latex

import numpy as np
from tqdm import tqdm
import ray
import time

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm

from acchamiltoniansandmatrices.Tracking.LinearMatrixTracking import nestList, GenerateNDimCoordinateGrid
from acchamiltoniansandmatrices.Hamiltonians.LatticeElementHamiltonians import HamDrift6D, HamQuad6D
from acchamiltoniansandmatrices.Hamiltonians.Operators import Poisson
from acchamiltoniansandmatrices.LieMaps.Poisson import PoissonBracket
from acchamiltoniansandmatrices.LieMaps.LieOperator import LieOperator
from acchamiltoniansandmatrices.Utils.Plotting import multi_countour_plot
from acchamiltoniansandmatrices.Factorization.Factorization import truncate, taylorize, dragt_finn_factorization
from acchamiltoniansandmatrices.Utils.Polynomials import getpoly, RotationMatrix2D, explicitCheck, geteigenvects
from acchamiltoniansandmatrices.Matrices.NumpyMatrices import RnpQuad6DChroma
from acchamiltoniansandmatrices.Matrices.SymbMatrices import RsymbQuad6DChroma
from acchamiltoniansandmatrices.Integrators import integrator

In [3]:
%matplotlib notebook
init_printing() 

# Jernej Track

In [4]:
def drift_poly(r,l):
    
    x = r[0]
    px = r[1]
    y = r[2]
    py = r[3]
    z = r[4]
    pz = r[5]
    
    xn = x + l * px + l * px**3 /2
    pxn = px
    yn = y 
    pyn = py
    zn = z 
    pzn = pz
    
    return xn, pxn, yn, pyn, zn, pzn


def TrackFodo(r0,duration=100):
    X_Exact = []
    X_poly = []



    for pos in r0:
        X0 = np.array(pos).reshape((-1, 6))

        pos = np.array(pos)
#         pos_poly = np.array(pos)
        
        X_Exact.append(pos)
#         X_poly.append(pos)

        for i in range(duration): 

            r = quadf(pos,_lqf,_k1)
            r = drift(r,_ld1)
            r = quadff(r,_lqd,_k2)
            r = drift(r,_ld2)
            r = quadf(r,_lqf,_k1)
            
#             r1 = quadf(pos_poly,_lqf,_k1)
#             r1 = drift_poly(r1,_ld1)
#             r1 = quadff(r1,_lqd,_k2)
#             r1 = drift_poly(r1,_ld2)
#             r1 = quadf(r1,_lqf,_k1)
            
            
            X_Exact.append(r)
#             X_poly.append(r1)
            
            pos = np.copy(r)
#             pos_poly = np.copy(r1)
            

    X_Exact = np.array(X_Exact)
#     X_poly = np.array(X_poly)
    
    return X_Exact#, X_poly

def drift(r,l):
    x = r[0]
    px = r[1]
    y = r[2]
    py = r[3]
    z = r[4]
    pz = r[5]
    
    d = np.sqrt((pz+1)**2 - px**2 - py**2)
    
    xn = x + l * px / d
    pxn = px
    yn = y + l * py / d
    pyn = py
    zn = z + l * (1-1/d) - pz*l/d
    pzn = pz
    
    return xn, pxn, yn, pyn, zn, pzn



def quadf(r,l,k):
    x = r[0]
    px = r[1]
    y = r[2]
    py = r[3]
    z = r[4]
    pz = r[5]
    
    k = np.abs(k)

    D = 1
    
    omega = np.sqrt(k)
    
    sin = np.sin(omega*l)
    cos = np.cos(omega*l)
    sinh = np.sinh(omega * l)
    cosh = np.cosh(omega * l)
    
    xn = cos * x + sin * px / (omega*D)
    pxn = cos * px - sin * (omega*D) * x
    yn = cosh * y + sinh * py / (omega*D)
    pyn = sinh * (omega*D) * y + cosh * py
    zn = z
    pzn = pz
    
    return xn, pxn, yn, pyn, zn, pzn

def quadff(r,l,k):
    x = r[0]
    px = r[1]
    y = r[2]
    py = r[3]
    z = r[4]
    pz = r[5]

    k = np.abs(k)
    D = 1
    
    omega = np.sqrt(k)
    sin = np.sin(omega*l)
    cos = np.cos(omega*l)
    sinh = np.sinh(omega * l)
    cosh = np.cosh(omega * l)
    
    xn = cosh * x + sinh * px / (omega*D)
    pxn = cosh * px + sinh * (omega*D) * x
    yn = cos * y + sin * py / (omega*D)
    pyn = cos * py - sin * (omega*D) * y
    zn = z
    pzn = pz    
    
    return xn, pxn, yn, pyn, zn, pzn

In [6]:
# Bifurcation setup for changing 1st drift space
_k1 = 1

_ld1 = 2.5 #outer islands appear


_ld2 = 1
_k2 = -_k1
_lqf = 0.1 #0.25
_lqd = 2*_lqf

dev = 0

# sub_val = list(zip([sym_ld2,sym_lqf, sym_lqd,sym_kqf,sym_kqd],[_ld2,_lqf,_lqd,_k1,_k2]))

_r0 = [[10e-7,0,0,0,0,dev],
       [10e-6,0,0,0,0,dev],
       [10e-5,0,0,0,0,dev],
       [10e-4,0,0,0,0,dev],
       [10e-3,0,0,0,0,dev],
       [10e-2,0,0,0,0,dev],
       [0.1,0,0,0,0,dev],
       [0.125,0,0,0,0,dev],
       [0.15,0,0,0,0,dev],
       [0.75,0,0,0,0,dev],
       [0.2,0,0,0,0,dev],
       [0.15,0,0,0,0,dev],
       [0.25,0,0,0,0,dev],
       [0.275,0,0,0,0,dev],
       [0.3,0,0,0,0,dev],
       [0.325,0,0,0,0,dev],
       [0.35,0,0,0,0,dev],
       [0.375,0,0,0,0,dev],
       [0.4,0,0,0,0,dev],
       [0.425,0,0,0,0,dev],
       [0.45,0,0,0,0,dev],
       [0.475,0,0,0,0,dev],
       [0.5,0,0,0,0,dev],
       [0.525,0,0,0,0,dev],
       [0.55,0,0,0,0,dev],
       [0.574,0,0,0,0,dev],
       [0.6,0,0,0,0,dev],
       [0.625,0,0,0,0,dev],
       [0.65,0,0,0,0,dev],
       [0.675,0,0,0,0,dev],
       [0.7,0,0,0,0,dev],
       [0.725,0,0,0,0,dev],
       [0.75,0,0,0,0,dev],
       [0.775,0,0,0,0,dev],
       [0.8,0,0,0,0,dev],
       [0.825,0,0,0,0,dev],
       [0.85,0,0,0,0,dev],
       [0.875,0,0,0,0,dev],
       [0.9,0,0,0,0,dev],
       [1,0,0,0,0,dev],
       [1.1,0,0,0,0,dev],]

print("ld1: ",_ld1)
print("ld2: ",_ld2)
print("lqf: ",_lqf)
print("lqd: ",_lqd)

print("k1: ",_k1)
print("k2: ",_k2)

ld1:  2.5
ld2:  1
lqf:  0.1
lqd:  0.2
k1:  1
k2:  -1


In [ ]:
variables = (sym_x, sym_px)
_ld_spec = np.arange(0.62,0.40,-0.02)
_k_spec = np.arange(0,2,0.05)

for l in _ld_spec:
    _k1  = 1
    _ld1 = l
    _ld2 = 1
    _k2  = -_k1
    _lqf = 0.5
    _lqd = 2*_lqf
    
    print("ld1: ",_ld1," ld2: ",_ld2, " k1: ",_k1," k2: ",_k2)

#     X_Exact, X_poly = TrackFodo(_r0,2000)
    X_Exact = TrackFodo(_r0,2000)

    # subplot grid
    fig = plt.figure(constrained_layout=True, figsize = (12,12))
    gs  = fig.add_gridspec(1, 1)

    ax1 = fig.add_subplot(gs[0,0])
    ax1.plot(X_Exact[:, 0], X_Exact[:, 1], '.k', alpha=0.1,label='Exact')

    
    
    var = [sym_ld1,sym_ld2,sym_lqf, sym_lqd, sym_kqf,sym_kqd]
    val = [_ld1,_ld2,_lqf,_lqd,_k1,_k2]
    
#     xdot = Poisson(sym_x,H_full,i_coord,i_mom).expand().subs(sub_val)
#     p1 = poly(xdot, sym_x,sym_px)
    
#     labels = []
#     for index,monomial in enumerate(p1.monoms()): #iterate over all monomials in taylor
#         mon = prod(a**b for a,b in zip(variables,monomial)) #reconstruct monomial
#         labels.append(mon)
        
#     ax2.set_xticks(np.arange(len(var)))
#     ax2.set_xticklabels(var)
    
#     ax2.plot(np.arange(len(var)), val, '.', markersize = 10,label='xdot')
    
    
    
#     pdot = Poisson(sym_px,H_full,i_coord,i_mom).expand().subs(sub_val)
#     p2 = poly(pdot, sym_x,sym_px)
    
#     labels = []
#     for index,monomial in enumerate(p1.monoms()): #iterate over all monomials in taylor
#         mon = prod(a**b for a,b in zip(variables,monomial)) #reconstruct monomial
#         labels.append(mon)
        
#     ax3.set_xticks(np.arange(len(p2.monoms())))
#     ax3.set_xticklabels(labels)
    
    
#     ax3.plot(np.arange(len(p2.monoms())), p2.coeffs(), '.', markersize = 10,label='pdot')


#     ax1.legend()
    
    title = "Drift Length: " + str(round(_ld1,2)) + " m"

    plt.suptitle(title)
#     plt.savefig('Bifurcation_DPG_' + str(round(_ld1,2)) + '.png')
#     plt.savefig('Bifurcation_DPG_Start.png')
    ax1.grid("on")
#     ax2.grid("on")
    plt.show()